# Math Tutor Assistant


## Objective

This notebook demonstrates the following:

- Showcases the foundational concepts of Assistants such as Threads, Messages, Runs, Tools, and lifecycle management.

This tutorial uses the following Azure AI services:

- Access to Azure OpenAI Service - you can apply for access [here](https://aka.ms/oai/access)
- Azure OpenAI service - you can create it from instructions [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource),
- Azure OpenAI Studio - go to [https://oai.azure.com/](https://oai.azure.com/) to work with the Assistants API Playground
- A connection to the Azure OpenAI Service with a [Key and Endpoint](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart)

Reference:

- Learn more about how to use Assistants with our [How-to guide on Assistants](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/assistant).

Reference:

- [Assistants OpenAI Overview](https://platform.openai.com/docs/assistants/overview)
- [Github-OpenAI Python/examples/Assistant demo notebook](https://github.com/openai/openai-python/blob/main/examples/assistant.py)


## Time

You should expect to spend 5-10 minutes running this sample.


## About this example

The objective of the provided Python file is to create an Azure OpenAI Assistant named "Math Tutor" using the Azure OpenAI API. The assistant is designed to function as a personal math tutor, capable of answering math questions through code interpretation. The script initiates a conversation with the assistant, guiding it through various mathematical queries and scenarios to showcase its capabilities.

This sample provides developers with a clear demonstration of how to integrate the Assistants API into their projects, highlighting its simplicity and effectiveness in leveraging foundational concepts.


## Before you begin


### Installation

Install the following packages required to execute this notebook.


In [ ]:
# Install the packages
%pip install -r ../requirements.txt

### Parameters


In [ ]:
import os

from dotenv import load_dotenv

load_dotenv("../.env")  # make sure to have the .env file in the root directory of the project
api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")

should_cleanup: bool = False

## Run this Example


## Load the required libraries


In [ ]:
import io
from datetime import datetime
from typing import Iterable
from PIL import Image

from openai import AzureOpenAI
from openai.types.beta.threads import Message, TextContentBlock, ImageFileContentBlock

### Create an Azure OpenAI client


In [ ]:
client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)

### Create an Assistant and a Thread


In [ ]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model=api_deployment_name,
)

thread = client.beta.threads.create()

### Format and display the Assistant Messages for text and images


In [ ]:
def format_messages(messages: Iterable[Message]) -> None:
    message_list = []

    # Get all the messages till the last user message
    for message in messages:
        message_list.append(message)
        if message.role == "user":
            break

    # Reverse the messages to show the last user message first
    message_list.reverse()

    # Print the user or Assistant messages or images
    for message in message_list:
        for item in message.content:
            # Determine the content type
            if isinstance(item, TextContentBlock):
                print(f"{message.role}:\n{item.text.value}\n")
            elif isinstance(item, ImageFileContentBlock):
                # Retrieve image from file id
                response_content = client.files.content(item.image_file.file_id)
                data_in_bytes = response_content.read()
                # Convert bytes to image
                readable_buffer = io.BytesIO(data_in_bytes)
                image = Image.open(readable_buffer)
                # Resize image to fit in terminal
                width, height = image.size
                image = image.resize((width // 2, height // 2), Image.LANCZOS)
                # Display image
                image.show()

### Process the user messages


In [ ]:
def process_message(content: str) -> None:
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=content)
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="Please address the user as Jane Doe. The user has a premium account. Be assertive, accurate, and polite. Ask if the user has further questions. "
        + "The current date and time is: "
        + datetime.now().strftime("%x %X")
        + ". ",
    )

    print("processing ...")
    completed_run = client.beta.threads.runs.poll(run_id=run.id, thread_id=thread.id)

    # Check final status and handle accordingly
    if completed_run.status == "completed":
        print("Run completed successfully.")
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        format_messages(messages)
    elif completed_run.status == "failed":
        print("Run failed.")
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        format_messages(messages)
    else:
        print(f"Unexpected run status: {completed_run.status}")

### Have a conversation with the Assistant


In [ ]:
process_message("What is the linear equation when two (x,y) points are (1,1) and (5,10)?")

In [ ]:
process_message("I need to solve the equation `3x + 11 = 14`. Can you help me?")

In [ ]:
process_message(
    """x=r*cos(u)sin(v)
y=r*sin(u)sin(v)
r=2+sin(7*u+5*v) 

for 0<u<2π and 0<v<π.

Create a graph of the equation z=r*cos(v)."""
)

## Cleaning up


In [ ]:
if should_cleanup:
    client.beta.assistants.delete(assistant.id)
    client.beta.threads.delete(thread.id)